# <a> Precificação de Aluguéis Indicium </a>

O objetivo do presente projeto consiste na construção de uma estratégia de precificação para aluguéis temporários na cidade de Nova York. Dessa forma, neste trabalho serão constrúidas duas análises, sendo a primeira de natureza diagnóstica e a última de caráter preditivo. 

Na primeira parte do projeto, será desenvolvida uma análise exploratória de dados que vise a estabelecer determinadas hipóteses de negócio. 

Em segundo plano, um modelo preditivo será construído a fim de auxiliar a Indicium elaborar a estratégia de precificação para o cliente que a contratou. 

In [1]:
# Instalando bibliotecas necessárias
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

In [3]:
# Configurando pandas para mostrar todas as linhas e colunas nos DataFrames
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# Configurando pandas para não mostrar notação científica 
pd.set_option("display.float_format", lambda x: '%.2f' % x)

# Ignorando possíveis Warnings
warnings.filterwarnings('ignore')

## <a> Dados </a>

Vamos começar lendo a base de dados e, posteriormente, criando um dicionário que represente o significado de cada feature do Data Frame.

In [4]:
# Lendo base de dados
df_precificacao = pd.read_csv("./data/teste_indicium_precificacao.csv")

In [5]:
# Visualizando 5 primeiras linhas do DataFrame
df_precificacao.head()

,id,nome,host_id,host_name,bairro_group,bairro,latitude,longitude,room_type,price,minimo_noites,numero_de_reviews,ultima_review,reviews_por_mes,calculado_host_listings_count,disponibilidade_365
0,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75,-73.98,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
1,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.81,-73.94,Private room,150,3,0,NaN,NaN,1,365
2,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.69,-73.96,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
3,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.80,-73.94,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
4,5099,Large Cozy 1 BR Apartment In Midtown East,7322,Chris,Manhattan,Murray Hill,40.75,-73.97,Entire home/apt,200,3,74,2019-06-22,0.59,1,129


In [6]:
# Checando se a coluna 'id' possui apenas valores únicos 
df_precificacao['id'].is_unique

True

Já que a coluna 'id' possui apenas valores únicos (referentes a cada anúncio nos dados do aplicativo), vamos transformá-la no índice do DataFrame.

In [13]:
# Passando 'id' como indíce do DataFrame
df_precificacao = df_precificacao.set_index("id")

# Visualizando novamente
df_precificacao.head()

,nome,host_id,host_name,bairro_group,bairro,latitude,longitude,room_type,price,minimo_noites,numero_de_reviews,ultima_review,reviews_por_mes,calculado_host_listings_count,disponibilidade_365
id,,,,,,,,,,,,,,,
2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75,-73.98,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.81,-73.94,Private room,150,3,0,NaN,NaN,1,365
3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.69,-73.96,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.80,-73.94,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
5099,Large Cozy 1 BR Apartment In Midtown East,7322,Chris,Manhattan,Murray Hill,40.75,-73.97,Entire home/apt,200,3,74,2019-06-22,0.59,1,129


## <a> Dicionário de Dados </a>

In [14]:
# Criando dicionário de dados
dicionario_dados = dict()

# Preenchendo dicionário de dados
dicionario_dados['nome'] = 'Representa o nome do anúncio'
dicionario_dados['host_id'] = 'Representa o id do usuário que hospedou o anúncio'
dicionario_dados['host_name'] = 'Contém o nome do usuário que hospedou o anúncio'
dicionario_dados['bairro_group'] = 'Contém o nome do bairro onde o anúncio está localizado'
dicionario_dados['bairro'] = 'Contém o nome da área onde o anúncio está localizado'
dicionario_dados['latitude'] = 'Contém a latitude do local'
dicionario_dados['longitude'] = 'Contém a longitude do local'
dicionario_dados['room_type'] = 'Contém o tipo de espaço de cada anúncio'
dicionario_dados['price'] = 'Contém o preço por noite em dólares listado pelo anfitrião'
dicionario_dados['minimo_noites'] = 'Contém o número mínimo de noites que o usuário deve reservar'
dicionario_dados['numero_de_reviews'] = 'Contém o número de comentários dados a cada listagem'
dicionario_dados['ultima_review'] = 'Contém a data da última revisão dada à listagem'
dicionario_dados['reviews_por_mes'] = 'Contém o número de avaliações fornecidas por mês'
dicionario_dados['calculado_host_listings_count'] = 'Contém a quantidade de listagem por host'
dicionario_dados['disponibilidade_365'] = 'Contém o número de dias em que o anúncio está disponível para reserva'

In [15]:
# Ao popular o dicionário de dados com a descrição de cada variável, se torna mais fácil acessar cada significado
dicionario_dados['bairro_group'] # Exemplo com feature 'bairro_group'

'Contém o nome do bairro onde o anúncio está localizado'

## <a> Manipulação de Dados Inicial </a>

In [16]:
# Verificando todas as colunas do DataFrame
df_precificacao.columns

Index(['nome', 'host_id', 'host_name', 'bairro_group', 'bairro', 'latitude',
       'longitude', 'room_type', 'price', 'minimo_noites', 'numero_de_reviews',
       'ultima_review', 'reviews_por_mes', 'calculado_host_listings_count',
       'disponibilidade_365'],
      dtype='object')

In [17]:
# Verificando número de features
df_precificacao.columns.nunique()

15

In [18]:
# Criando função para mostrar dimensões, tipos de dados de cada feature e demais informações do DataFrame
def informacao(dataframe):
    print(f"As dimensões do DataFrame são: {(dataframe.shape[0])} linhas e {(dataframe.shape[1])} colunas")
    print("\n")
    print(f"As features do DataFrame e seus tipos são:\n\n{dataframe.dtypes}")
    print("\n")
    print("Demais informações sobre o DataFrame: \n")
    dataframe.info()

In [19]:
# Chamando função para o DataFrame 'df_precificacao'
informacao(df_precificacao)

As dimensões do DataFrame são: 48894 linhas e 15 colunas


As features do DataFrame e seus tipos são:

nome                              object
host_id                            int64
host_name                         object
bairro_group                      object
bairro                            object
latitude                         float64
longitude                        float64
room_type                         object
price                              int64
minimo_noites                      int64
numero_de_reviews                  int64
ultima_review                     object
reviews_por_mes                  float64
calculado_host_listings_count      int64
disponibilidade_365                int64
dtype: object


Demais informações sobre o DataFrame: 

<class 'pandas.core.frame.DataFrame'>
Index: 48894 entries, 2595 to 36487245
Data columns (total 15 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  ----

> A partir da visualização dos tipos de dados de cada feature do Pandas DataFrame original, é possível ter uma noção de quais variáveis categóricas (object) deveremos codificar para a construção do modelo preditivo. As variáveis 'room_type' e 'bairro_group' são exemplos de variáveis categóricas que podem ser codificadas posteriormente.

Outro ponto a se investigar para codificação de variáveis é a cardinalidade de cada coluna, que pode dar indícios de qual é a melhor abordagem para essa tarefa.

In [81]:
# Checando cardinalidade de cada coluna
df_precificacao.nunique()

nome                             47904
host_id                          37457
host_name                        11452
bairro_group                         5
bairro                             221
latitude                         19048
longitude                        14718
room_type                            3
price                              674
minimo_noites                      109
numero_de_reviews                  394
ultima_review                     1764
reviews_por_mes                    937
calculado_host_listings_count       47
disponibilidade_365                366
dtype: int64

>  Outra informação interessante é referente aos valores não nulos de cada feature. Percebe-se que nas colunas 'nome' e 'host_name' há poucos valores faltantes, enquanto em 'ultima_review' e 'reviews_por_mes' o número de valores faltantes é considerável.

Em razão disso, vamos criar uma função que mostre cada feature do DataFrame e sua correspondente porcentagem de valores faltantes. Isso pode ser bastante útil para analisar qual estratégia de imputação será utilizada (se a imputação de valores for cabível).

In [79]:
# Definindo função para mostrar valores faltantes e calcular percentagem de cada um referente ao total de linhas do DataFrame
def pct_missing(df):
    # Atribuindo array de valores faltantes a variavel 'missing'
    missing = df_precificacao.isnull().sum()
    
    # Mostrando valores faltantes por feature
    print("Esses são os valores faltantes para cada feature: ")
    print("\n") 
    print(missing)
    print("\n") 
    
    # Definindo dicionario 'pct' para armazenar estrutura de chave:valor para nome da feature e porcentagem de missing data
    pct = {}
    
    # List comprehension e loop for para dividir os valores faltantes de cada feature pelo total de linhas para cada feature
    for feature in missing.index:
        pct[feature] = (f"{round((missing[feature] / df_precificacao.shape[0]) * 100, 2)}%")
    print("Essas são as porcentagens de valores faltantes para cada feature: ")
    print("\n")
    
    # Retornando dicionário que mostra cada feature e sua porcentagem de missing data 
    return pct

In [80]:
pct_missing(df_precificacao)

Esses são os valores faltantes para cada feature: 


nome                                16
host_id                              0
host_name                           21
bairro_group                         0
bairro                               0
latitude                             0
longitude                            0
room_type                            0
price                                0
minimo_noites                        0
numero_de_reviews                    0
ultima_review                    10052
reviews_por_mes                  10052
calculado_host_listings_count        0
disponibilidade_365                  0
dtype: int64


Essas são as porcentagens de valores faltantes para cada feature: 




{'nome': '0.03%',
 'host_id': '0.0%',
 'host_name': '0.04%',
 'bairro_group': '0.0%',
 'bairro': '0.0%',
 'latitude': '0.0%',
 'longitude': '0.0%',
 'room_type': '0.0%',
 'price': '0.0%',
 'minimo_noites': '0.0%',
 'numero_de_reviews': '0.0%',
 'ultima_review': '20.56%',
 'reviews_por_mes': '20.56%',
 'calculado_host_listings_count': '0.0%',
 'disponibilidade_365': '0.0%'}

Levando em consideração as primeiras impressões de nossa base de dados, vamos dar início a uma etapa muito importante em qualquer projeto de Ciência de Dados, a EDA.

## <a> Análise Exploratória de Dados </a>

Além da capacidade investigativa inicial que uma boa EDA proporciona ao Cientista de Dados, a Análise Exploratória de Dados também é importante para propor hipóteses de negócio e introduzir a estratégia de modelagem preditiva que será utilizada. Ao verificar distribuições, medidas de tendência central, separatrizes, frequência e correlação entre as variáveis, pode-se perceber quais são os modelos que obterão melhor desempenho na solução do problema de negócio com que nos deparamos.

Vamos começar a análise com o método describe, selecionando apenas colunas de valores inteiros que sejam relevantes para a modelagem.

In [95]:
# Criando lista das features relevantes
features_relevantes = ['price', 'minimo_noites', 'numero_de_reviews', 
                       'reviews_por_mes', 'calculado_host_listings_count', 'disponibilidade_365']

# Filtrando o describe mostrando apenas as colunas relevantes
(df_precificacao.describe()).loc[:, features_relevantes]

,price,minimo_noites,numero_de_reviews,reviews_por_mes,calculado_host_listings_count,disponibilidade_365
count,48894.00,48894.00,48894.00,38842.00,48894.00,48894.00
mean,152.72,7.03,23.27,1.37,7.14,112.78
std,240.16,20.51,44.55,1.68,32.95,131.62
min,0.00,1.00,0.00,0.01,1.00,0.00
25%,69.00,1.00,1.00,0.19,1.00,0.00
50%,106.00,3.00,5.00,0.72,1.00,45.00
75%,175.00,5.00,24.00,2.02,2.00,227.00
max,10000.00,1250.00,629.00,58.50,327.00,365.00


In [100]:
# Analisando a moda do preço por noite 
df_precificacao['price'].mode()

0    100
Name: price, dtype: int64

In [151]:
# Checando o número de vezes que a variável minimo_noites assume o valor máximo de 1250
df_precificacao.loc[df_precificacao['minimo_noites'] == 1250].shape[0]

1

In [173]:
# Checando o número de aluguéis com menos de 10 reviews
num_10 = df_precificacao.loc[df_precificacao['numero_de_reviews'] < 10].shape[0]
print(num_10)

# Calculando porcentagem de apartamentos com menos de 10 reviews
print(f'{round((num_10/df_precificacao.shape[0]) * 100, 2)}% dos apartamentos possuem menos de 10 reviews')

29519
60.37% dos apartamentos possuem menos de 10 reviews


In [214]:
# Checando o número de aluguéis disponíveis por menos de 50 dias no ano
num_50 = df_precificacao.loc[df_precificacao['disponibilidade_365'] < 10].shape[0]
print(num_50)

# Calculando porcentagem de apartamentos disponíveis por menos de 50 dias no ano
print(f'{round((num_50/df_precificacao.shape[0]) * 100, 2)}% dos apartamentos está disponível por menos de 10 dias no ano')

19980
40.86% dos apartamentos está disponível por menos de 10 dias no ano


Levando em consideração as informações expressas acima, é possível observar que:

* A média do preço por noite dos aluguéis ('price') é maior que sua mediana e sua moda, o que revela uma assimetria à direita na distribuição dessa variável.
* Na variável minimo_noites, relativa ao número mínimo de noites que o usuário deve reservar em determinado aluguel, percebe-se que o valor máximo é de 1250 noites, representando apenas uma observação em todo conjunto de dados.
* O número de reviews geralmente é baixo para os apartamentos. É possível perceber que 50% dos aluguéis possuem menos de 5 reviews e 60,37% deles possui menos de 10 reviews.
* Existem muitos apartamentos que ficam disponíveis para aluguel por pouco tempo no ano. Cerca de 50% dos aluguéis estão disponíveis por menos de 45 dias no ano. Além disso, é interessante perceber que quase 41% dos apartamentos ficam disponíveis por apenas 1 dia no ano.

# COMEÇAR A PARTE GRÁFICA 